<a href="https://colab.research.google.com/github/TeePilzz/ml-portfolio/blob/main/wildlife-monitoring-system/Eco_Monitoring_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌲 Проект: Мониторинг экосистемы с помощью Computer Vision

**Цель:** Автоматизация процесса классификации диких животных на изображениях с фотоловушек.

**Задача:** Использовать предобученную нейросеть для распознавания видов животных из датасета *Wildlife Animals Images*.

**Технологии:**
* `Transformers` (Microsoft ResNet-18)
* `KaggleHub` (загрузка данных)
* `Pandas` (анализ результатов)

In [4]:
# Установка необходимых библиотек (раскомментировать при необходимости)
# !pip install -q kagglehub transformers scikit-learn pandas pillow

import os
import kagglehub
from transformers import pipeline
from PIL import Image
import pandas as pd
from sklearn.metrics import accuracy_score

print("Библиотеки импортированы успешно.")

Библиотеки импортированы успешно.


In [5]:
# 1. Загрузка данных
# Используем KaggleHub для автоматической загрузки актуальной версии датасета
print("Загрузка датасета...")
dataset_path = kagglehub.dataset_download("anshulmehtakaggl/wildlife-animals-images")

print(f"Датасет загружен в папку: {dataset_path}")

Загрузка датасета...
Using Colab cache for faster access to the 'wildlife-animals-images' dataset.
Датасет загружен в папку: /kaggle/input/wildlife-animals-images


In [6]:
# 2. Инициализация модели
# Используем ResNet-18: легкая и быстрая модель, идеально подходящая
# для запуска на периферийных устройствах (например, Raspberry Pi в лесу)
print("\nИнициализация модели (ResNet-18)...")
classifier = pipeline("image-classification", model="microsoft/resnet-18")


Инициализация модели (ResNet-18)...


Device set to use cpu


In [7]:
# 3. Анализ данных и прогон через модель
true_labels = []   # Истинные метки (из названия папки)
pred_labels = []   # Предсказания нейросети
report_data = []   # Данные для итогового отчета

print("Начинаем мониторинг...")

# Обход директорий датасета
for root, dirs, files in os.walk(dataset_path):
    folder_name = os.path.basename(root)

    # Фильтрация данных:
    # Работаем только с папками 224px (оптимальный размер для ResNet-18)
    if "resize-224" not in folder_name:
        continue

    # Парсинг метки класса:
    # Превращаем "cheetah-resize-224" -> "cheetah" для сравнения
    correct_animal_name = folder_name.split('-')[0]

    print(f"Обработка вида: {correct_animal_name}...")

    # Фильтруем только изображения
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Ограничение выборки: берем первые 5 фото каждого вида для быстрой валидации
    for img_file in image_files[:5]:
        try:
            # Загрузка и конвертация изображения
            img_path = os.path.join(root, img_file)
            image = Image.open(img_path).convert('RGB')

            # Получение предсказания от AI
            result = classifier(image)[0]
            ai_prediction = result['label'].lower()
            confidence = result['score']

            # Валидация: считается верным, если имя животного есть в ответе модели
            is_correct = correct_animal_name in ai_prediction

            # Сохранение результатов
            true_labels.append(correct_animal_name)
            pred_labels.append(correct_animal_name if is_correct else ai_prediction)

            report_data.append({
                "Файл": img_file,
                "Факт": correct_animal_name,
                "Ответ AI": ai_prediction,
                "Статус": "1" if is_correct else "0" # 1 - верно, 0 - ошибка
            })

        except Exception as e:
            # Пропускаем битые файлы
            continue

Начинаем мониторинг...
Обработка вида: fox...
Обработка вида: fox...
Обработка вида: lion...
Обработка вида: lion...
Обработка вида: hyena...
Обработка вида: resize...
Обработка вида: wolf...
Обработка вида: wolf...
Обработка вида: tiger...
Обработка вида: tiger...
Обработка вида: cheetah...
Обработка вида: resize...


In [8]:
# 4. Формирование отчета и расчет метрик
if len(true_labels) > 0:
    accuracy = accuracy_score(true_labels, pred_labels)

    print("\nРезультаты мониторинга")

    # Создаем DataFrame для наглядности
    df = pd.DataFrame(report_data)

    # Выводим случайную выборку из 10 проверенных изображений
    display(df.sample(min(len(df), 10)))

    print(f"\nИтоговая точность (Accuracy): {accuracy*100:.2f}%")

    # Краткий вывод для системы принятия решений
    if accuracy > 0.8:
        print("Вывод: Система готова к эксплуатации.")
    else:
        print("Вывод: Требуется дообучение модели.")
else:
    print("Файлы не найдены. Проверь, что датасет скачался корректно.")


Результаты мониторинга


,Файл,Факт,Ответ AI,Статус
26,00000137_224resized.png,resize,"cheetah, chetah, acinonyx jubatus",0
2,00000394_224resized.png,fox,"red fox, vulpes vulpes",1
15,00000137_224resized.png,wolf,"timber wolf, grey wolf, gray wolf, canis lupus",1
17,00000146_224resized.png,wolf,"timber wolf, grey wolf, gray wolf, canis lupus",1
10,00000464_224resized.png,resize,"hyena, hyaena",0
13,00000281_224resized.png,resize,"hyena, hyaena",0
7,00000146_224resized.png,lion,"lion, king of beasts, panthera leo",1
5,00000394_224resized.png,lion,"lion, king of beasts, panthera leo",1
1,00000137_224resized.png,fox,"red fox, vulpes vulpes",1
14,00000336_224resized.png,resize,"hyena, hyaena",0



Итоговая точность (Accuracy): 66.67%
Вывод: Требуется дообучение модели.
